In [1]:
import math
import torch
from torch import nn
from torch.nn.parameter import Parameter
from transformers import AutoModel

/home/hoang/miniconda3/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Implement LoRA layer

In [20]:
class LoRALayer(nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super(LoRALayer, self).__init__()
        self.A = Parameter(torch.empty(in_dim, rank))
        nn.init.kaiming_uniform_(self.A, a=math.sqrt(5)) # follow: https://github.com/microsoft/LoRA/blob/main/loralib/layers.py#L124
        self.B = Parameter(torch.zeros(rank, out_dim))
        self.alpha = alpha

    def forward(self, x):
        x = self.alpha * (x @ self.A @ self.B)
        return x


class LinearWithLoRA(nn.Module):
    def __init__(self, linear, rank, alpha):
        super(LinearWithLoRA, self).__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        return self.linear(x) + self.lora(x)

# Function to replace `Linear` layers with `LinearWithLoRA` layers

In [21]:
def replace_linear_with_lora(model, rank, alpha):
    for name, module in model.named_children():
        if isinstance(module, nn.Linear):
            # Check if the module is an instance of torch.nn.Linear
            # Replace the Linear layer with a LinearWithLoRA layer
            setattr(model, name, LinearWithLoRA(module, rank, alpha))
        else:
            # If the module is not a Linear layer, recursively apply the function to the child modules
            replace_linear_with_lora(module, rank, alpha)

# Apply LoRA Layers

In [22]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import os
os.environ['CURL_CA_BUNDLE'] = ''

proxy = 'http://192.168.5.8:3128'
os.environ['HTTP_PROXY'] = proxy
os.environ['HTTPS_PROXY'] = proxy

model = AutoModel.from_pretrained('microsoft/deberta-v3-large', cache_dir='../cache/')

/home/hoang/miniconda3/envs/llm/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.5.8'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/hoang/miniconda3/envs/llm/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.5.8'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


## Calculate total number of trainable parameters of the original model

In [23]:
total_params_no_lora = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params_no_lora

434012160

## Freeze All Layers Before Applying LoRA Layers

In [24]:
for param in model.parameters():
    param.requires_grad = False

In [25]:
total_trainable_params_after_freezing = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_trainable_params_after_freezing

0

## Replace Linear Layers

In [26]:
replace_linear_with_lora(model=model, rank=8, alpha=16)

In [27]:
total_params_with_lora = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params_with_lora

3538944

In [19]:
((total_params_no_lora - total_params_with_lora) / total_params_no_lora) * 100

99.1845979614949

Thus, applying LoRA reduces the number of trainable parameters by approximately 99.18%.

By applying LoRA, we are able to train the model with only about 1% of the original number of parameters. This massive reduction in the number of trainable parameters (approximately 99.18%) means that we can achieve efficient fine-tuning with significantly fewer resources, making the process much more efficient in terms of computation and memory usage.